# **CSE 7324 Lab 3: Extending Logistic Regression**
### *Thomas Adams, Suleiman Hijazeen, Nancy Le and Andrew Whigham*
------

### **1. Preparation and Overview**
------

In [366]:
# dependencies for lab 1
import pandas as pd
import numpy as np
import missingno as msno
# use plotly in offline mode to not have active connection to plotly servers
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import warnings
#warnings.simplefilter('ignore', category=DeprecationWarning)
warnings.filterwarnings("ignore")
%matplotlib inline


In [416]:
# import data
shelter_outcomes = pd.read_csv("C:/Users/sulem/OneDrive/Desktop/machin learnign/Project3/aac_shelter_cat_outcome_eng.csv")
# filter animal type for just cats
cats = shelter_outcomes[shelter_outcomes['animal_type'] == 'Cat']
#print(cats.head())

# remove age_upon_outcome and recalculate to standard units (days)
age = cats.loc[:,['datetime', 'date_of_birth']]
# convert to datetime
age.loc[:,'datetime'] = pd.to_datetime(age['datetime'])
age.loc[:,'date_of_birth'] = pd.to_datetime(age['date_of_birth'])
# calculate cat age in days
cats.loc[:,'age'] = (age.loc[:,'datetime'] - age.loc[:,'date_of_birth']).dt.days
# get dob info
cats['dob_month'] = age.loc[:, 'date_of_birth'].dt.month
cats['dob_day'] = age.loc[:, 'date_of_birth'].dt.day
cats['dob_dayofweek'] = age.loc[:, 'date_of_birth'].dt.dayofweek
# get month from datetime
cats['month'] = age.loc[:,'datetime'].dt.month
# get day of month
cats['day'] = age.loc[:,'datetime'].dt.day
# get day of week
cats['dayofweek'] = age.loc[:, 'datetime'].dt.dayofweek
# get hour of day
cats['hour'] = age.loc[:, 'datetime'].dt.hour
# get quarter
cats['quarter'] = age.loc[:, 'datetime'].dt.quarter

# clean up breed attribute
# get breed attribute for processing
# convert to lowercase, remove mix and strip whitespace
# remove space in 'medium hair' to match 'longhair' and 'shorthair'
# split on either space or '/'
breed = cats.loc[:, 'breed'].str.lower().str.replace('mix', '').str.replace('medium hair', 'mediumhair').str.strip().str.split('/', expand=True)
cats['breed'] = breed[0]
cats['breed1'] = breed[1]

# clean up color attribute
# convert to lowercase
# strip spaces
# split on '/'
color = cats.loc[:, 'color'].str.lower().str.strip().str.split('/', expand=True)
cats['color'] = color[0]
cats['color1'] = color[1]

# clean up sex_upon_outcome
sex = cats['sex_upon_outcome'].str.lower().str.strip().str.split(' ', expand=True)
sex[0].replace('spayed', True, inplace=True)
sex[0].replace('neutered', True, inplace=True)
sex[0].replace('intact', False, inplace=True)
sex[1].replace(np.nan, 'unknown', inplace=True)
cats['spayed_neutered'] = sex[0]
cats['sex'] = sex[1]

# add in domesticated attribute
cats['domestic'] = np.where(cats['breed'].str.contains('domestic'), 1, 0)

# combine outcome and outcome subtype into a single attribute
cats['outcome_subtype'] = cats['outcome_subtype'].str.lower().str.replace(' ', '-').fillna('unknown')
cats['outcome_type'] = cats['outcome_type'].str.lower().str.replace(' ', '-').fillna('unknown')
cats['outcome'] = cats['outcome_type'] + '_' + cats['outcome_subtype']

# drop unnecessary columns
cats.drop(columns=['animal_id', 'name', 'animal_type', 'age_upon_outcome', 'date_of_birth', 'datetime', 'monthyear', 'sex_upon_outcome', 'outcome_subtype', 'outcome_type'], inplace=True)
#print(cats['outcome'].value_counts())

cats.head()

,breed,color,count,sex,Spay/Neuter,Periods,Period Range,outcome_age_(days),outcome_age_(years),Cat/Kitten (outcome),...,dob_day,dob_dayofweek,month,day,dayofweek,hour,quarter,spayed_neutered,domestic,outcome
0,domestic shorthair,orange,1,male,No,2,7,14,0.038356,Kitten,...,7,0,7,22,1,16,3,False,1,transfer_partner
1,domestic shorthair,blue,1,female,No,1,30,30,0.082192,Kitten,...,16,0,8,14,3,18,3,False,1,adoption_unknown
2,domestic shorthair,white,1,female,Yes,3,30,90,0.246575,Kitten,...,26,2,6,29,6,17,2,True,1,adoption_offsite
3,domestic mediumhair,black,1,female,Yes,1,365,365,1.000000,Cat,...,27,2,3,28,4,14,1,True,1,return-to-owner_unknown
4,domestic shorthair,black,1,male,No,3,7,21,0.057534,Kitten,...,16,0,1,9,3,19,1,False,1,transfer_partner


In [417]:
cats.drop(columns=['breed1'], inplace=True)
cats.drop(columns=['color2'], inplace=True)
cats.drop(columns=['breed2'], inplace=True)
cats.drop(columns=['coat_pattern'], inplace=True)
cats.drop(columns=['Cat/Kitten (outcome)'], inplace=True)
cats.drop(columns=['sex_age_outcome'], inplace=True)
cats.drop(columns=['dob_monthyear'], inplace=True)
cats.drop(columns=['outcome_weekday'], inplace=True)
#Breed, Color, Color1, Spayed_Netured and Sex attributes need to be one hot encoded
cats_ohe = pd.get_dummies(cats, columns=['breed', 'color', 'color1', 'spayed_neutered', 'sex','Spay/Neuter','age_group','coat','domestic_breed','cfa_breed' ])
#out_t={'euthanasia_rabies-risk' : 1, 'unknown_unknown' : 2, 'adoption_barn' : 3, 'died_unknown' : 4, 'adoption_offsite' : 5, 'adoption_unknown' : 6, 'missing_in-foster' : 7, 'rto-adopt_unknown' : 8, 'died_enroute' : 9, 'died_in-surgery' : 10, 'transfer_snr' : 11, 'euthanasia_medical' : 12, 'euthanasia_aggressive' : 13, 'transfer_scrp' : 15, 'euthanasia_unknown' : 14, 'missing_unknown' : 16, 'died_in-foster' : 17, 'missing_possible-theft' : 18, 'adoption_foster' : 19, 'euthanasia_at-vet' : 20, 'missing_in-kennel' : 21, 'died_at-vet' : 22, 'transfer_partner' : 23, 'return-to-owner_unknown' : 25, 'disposal_unknown' : 24, 'euthanasia_underage' : 26, 'died_in-kennel' : 27, 'euthanasia_suffering' : 28, 'transfer_barn' : 29}
#out_t={'euthanasia_rabies-risk' : 1, 'unknown_unknown' : 2, 'adoption_barn' : 3, 'died_unknown' : 4, 'adoption_offsite' : 5, 'adoption_unknown' : 6, 'missing_in-foster' : 7, 'rto-adopt_unknown' : 8, 'died_enroute' : 9, 'died_in-surgery' : 10, 'transfer_snr' : 11, 'euthanasia_medical' : 12, 'euthanasia_aggressive' : 13, 'transfer_scrp' : 15, 'euthanasia_unknown' : 14, 'missing_unknown' : 16, 'died_in-foster' : 0, 'missing_possible-theft' : 0, 'adoption_foster' : 0, 'euthanasia_at-vet' : 0, 'missing_in-kennel' : 0, 'died_at-vet' : 0, 'transfer_partner' : 0, 'return-to-owner_unknown' : 0, 'disposal_unknown' : 0, 'euthanasia_underage' : 0, 'died_in-kennel' : 0, 'euthanasia_suffering' : 0, 'transfer_barn' : 0}
out_t={'euthanasia_suffering' : 0, 'died_in-kennel' : 0, 'return-to-owner_unknown' : 0, 'transfer_partner' : 1, 'euthanasia_at-vet' : 2, 'adoption_foster' : 3, 'died_in-foster' : 0, 'transfer_scrp' : 4, 'euthanasia_medical' : 0, 'transfer_snr' : 0, 'died_enroute' : 0, 'rto-adopt_unknown' : 0, 'missing_in-foster' : 0, 'adoption_offsite' : 0, 'adoption_unknown' :5,'euthanasia_rabies-risk' : 0, 'unknown_unknown' : 0, 'adoption_barn' : 0, 'died_unknown' : 0, 'died_in-surgery' : 0, 'euthanasia_aggressive' : 0, 'euthanasia_unknown' : 0, 'missing_unknown' : 0, 'missing_in-kennel' : 0, 'missing_possible-theft' : 0, 'died_at-vet' : 0, 'disposal_unknown' : 0, 'euthanasia_underage' : 0, 'transfer_barn' : 0}

cats_ohe.head()

# separate outcome from data
outcome = cats_ohe['outcome']
cats_ohe.drop(columns=['outcome'])

# split the data
X_train, X_test, y_train, y_test = train_test_split(cats_ohe, outcome, test_size=0.2, random_state=0)
X_train.drop(columns=['outcome'], inplace=True)
y_train = [out_t[item] for item in y_train]
#print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [436]:
import numpy as np
class BinaryLogisticRegressionBase:
    # private:
    def __init__(self, eta, iterations, C):
        self.eta = eta
        self.iters = iterations
        self.C=C
        # internally we will store the weights as self.w_ to keep with sklearn conventions
    
    def __str__(self):
        return 'Base Binary Logistic Regression Object, Not Trainable'
    
    # convenience, private and static:
    @staticmethod
    def _sigmoid(theta):
        return 1/(1+np.exp(-theta)) 
    
    @staticmethod
    def _add_bias(X):
        return np.hstack((np.ones((X.shape[0],1)),X)) # add bias term
    
    # public:
    def predict_proba(self,X,add_bias=True):
        # add bias term if requested
        Xb = self._add_bias(X) if add_bias else X
        return self._sigmoid(Xb @ self.w_) # return the probability y=1
    
    def predict(self,X):
        return (self.predict_proba(X)>0.5) #return the actual prediction
    
    # inherit from base class
class BinaryLogisticRegression(BinaryLogisticRegressionBase):
    #private:
    def __str__(self):
        if(hasattr(self,'w_')):
            return 'Binary Logistic Regression Object with coefficients:\n'+ str(self.w_) # is we have trained the object
        else:
            return 'Untrained Binary Logistic Regression Object'
        
    def _get_gradient(self,X,y):
        # programming \sum_i (yi-g(xi))xi
        gradient = np.zeros(self.w_.shape) # set gradient to zero
        for (xi,yi) in zip(X,y):
            # the actual update inside of sum
            gradi = (yi - self.predict_proba(xi,add_bias=False))*xi 
            # reshape to be column vector and add to gradient
            gradient += gradi.reshape(self.w_.shape) 
        
        return gradient/float(len(y))
       
    # public:
    def fit(self, X, y):
        Xb = self._add_bias(X) # add bias term
        num_samples, num_features = Xb.shape
        
        self.w_ = np.zeros((num_features,1)) # init weight vector to zeros
        
        # for as many as the max iterations
        for _ in range(self.iters):
            gradient = self._get_gradient(Xb,y)
            self.w_ += gradient*self.eta # multiply by learning rate 
import numpy as np
from scipy.special import expit

class VectorBinaryLogisticRegression(BinaryLogisticRegression):
    # inherit from our previous class to get same functionality
    @staticmethod
    def _sigmoid(theta):
        # increase stability, redefine sigmoid operation
        return expit(theta) #1/(1+np.exp(-theta))
    
    # but overwrite the gradient calculation
    def _get_gradient(self,X,y):
        ydiff = y-self.predict_proba(X,add_bias=False).ravel() # get y difference
        gradient = np.mean(X * ydiff[:,np.newaxis], axis=0) # make ydiff a column vector and multiply through
        
        return gradient.reshape(self.w_.shape)
    
    
from scipy.optimize import minimize_scalar
import copy
class LineSearchLogisticRegression(VectorBinaryLogisticRegression):
    
    # define custom line search for problem
    
    @staticmethod
    def objective_function(eta,X,y,w,grad,C=0.001):
        wnew = w - grad*eta
        g = expit(X @ wnew)
        return -np.sum(np.log(g[y==1]))-np.sum(np.log(1-g[y==0])) + C*sum(wnew**2)
    
        
    def fit(self, X, y):
        Xb = self._add_bias(X) # add bias term
        num_samples, num_features = Xb.shape
        
        self.w_ = np.zeros((num_features,1)) # init weight vector to zeros
        
        # for as many as the max iterations
        for _ in range(self.iters):
            gradient = -self._get_gradient(Xb,y)
            # minimization inopposite direction
            
            # do line search in gradient direction, using scipy function
            opts = {'maxiter':self.iters/50} # unclear exactly what this should be
            res = minimize_scalar(self.objective_function, # objective function to optimize
                                  bounds=(self.eta/1000,self.eta*10), #bounds to optimize
                                  args=(Xb,y,self.w_,gradient,0.001), # additional argument for objective function
                                  method='bounded', # bounded optimization for speed
                                  options=opts) # set max iterations
            
            eta = res.x # get optimal learning rate
            self.w_ -= gradient*eta # set new function values
            # subtract to minimize
class StochasticLogisticRegression(BinaryLogisticRegression):
    # stochastic gradient calculation 
    def _get_gradient(self,X,y):
        idx = int(np.random.rand()*len(y)) # grab random instance
        ydiff = y[idx]-self.predict_proba(X[idx],add_bias=False) # get y difference (now scalar)
        gradient = X[idx] * ydiff[:,np.newaxis] # make ydiff a column vector and multiply through
        
        gradient = gradient.reshape(self.w_.shape)
        gradient[1:] += -2 * self.w_[1:] * self.C
        
        return gradient
    
from scipy.optimize import fmin_bfgs
class BFGSBinaryLogisticRegression(BinaryLogisticRegression):
    
    @staticmethod
    def objective_function(w,X,y,C):
        g = expit(X @ w)
        return -np.sum(np.log(g[y==1]))-np.sum(np.log(1-g[y==0])) + C*sum(w**2) #-np.sum(y*np.log(g)+(1-y)*np.log(1-g))

    @staticmethod
    def objective_gradient(w,X,y,C):
        g = expit(X @ w)
        ydiff = y-g # get y difference
        gradient = np.mean(X * ydiff[:,np.newaxis], axis=0)
        gradient = gradient.reshape(w.shape)
        gradient[1:] += -2 * w[1:] * C
        return -gradient
    
    # just overwrite fit function
    def fit(self, X, y):
        
        Xb = self._add_bias(X) # add bias term
        num_samples, num_features = Xb.shape
        
        self.w_ = fmin_bfgs(self.objective_function, # what to optimize
                            np.zeros((num_features,1)), # starting point
                            fprime=self.objective_gradient, # gradient function
                            args=(Xb,y,self.C), # extra args for gradient and objective function
                            gtol=1e-03, # stopping criteria for gradient, |v_k|
                            maxiter=self.iters, # stopping criteria iterations
                            disp=False)
        
        self.w_ = self.w_.reshape((num_features,1))    
        
        
from numpy.linalg import pinv
class HessianBinaryLogisticRegression(BinaryLogisticRegression):
    # just overwrite gradient function
    def _get_gradient(self,X,y):
        g = self.predict_proba(X,add_bias=False).ravel() # get sigmoid value for all classes
        hessian = X.T @ np.diag(g*(1-g)) @ X - 2 * self.C # calculate the hessian

        ydiff = y-g # get y difference
        gradient = np.sum(X * ydiff[:,np.newaxis], axis=0) # make ydiff a column vector and multiply through
        gradient = gradient.reshape(self.w_.shape)
        gradient[1:] += -2 * self.w_[1:] * self.C
        
        return pinv(hessian) @ gradient

In [437]:
from scipy.optimize import minimize_scalar
import copy
class LogisticRegression:
    def __init__(self, eta, iterations,solver='leaner', C=0.001):
        self.eta = eta
        self.iters = iterations
        self.slv  = solver
        self.C=C
        # internally we will store the weights as self.w_ to keep with sklearn conventions
    
    def __str__(self):
        if(hasattr(self,'w_')):
            return 'MultiClass Logistic Regression Object with coefficients:\n'+ str(self.w_) # is we have trained the object
        else:
            return 'Untrained MultiClass Logistic Regression Object'
        
    def fit(self,X,y):
        num_samples, num_features = X.shape
        self.unique_ = np.sort(np.unique(y)) # get each unique class value
        num_unique_classes = len(self.unique_)
        self.classifiers_ = [] # will fill this array with binary classifiers
        
        for i,yval in enumerate(self.unique_): # for each unique value
            y_binary = (y==yval) # create a binary problem
            # train the binary classifier for this class
            if self.slv=='stochastic':
             slr = StochasticLogisticRegression(self.eta,self.iters,self.C)
             slr.fit(X,y_binary)
             self.classifiers_.append(slr)
            if self.slv=='steepest':
             mls=LineSearchLogisticRegression(self.eta,self.iters,self.C)
             mls.fit(X,y_binary)
             self.classifiers_.append(mls)
            if self.slv=='leaner':
             blr = VectorBinaryLogisticRegression(self.eta,self.iters)
             blr.fit(X,y_binary)
             self.classifiers_.append(blr)
            if self.slv=='newton':
             bfgslr = BFGSBinaryLogisticRegression(self.eta,self.iters,self.C)
             bfgslr.fit(X,y_binary)
             self.classifiers_.append(bfgslr)
            if self.slv=='newton1':
             newt = HessianBinaryLogisticRegression(self.eta,self.iters,self.C)
             newt.fit(X,y_binary)
             self.classifiers_.append(newt)
            
            # add the trained classifier to the list      
        # save all the weights into one matrix, separate column for each class
        self.w_ = np.hstack([x.w_ for x in self.classifiers_]).T
    
    
    
    def predict_proba(self,X):
        probs = []
        for blr in self.classifiers_:
            probs.append(blr.predict_proba(X)) # get probability for each classifier
        
        return np.hstack(probs) # make into single matrix
    
    def predict(self,X):
        return np.argmax(self.predict_proba(X),axis=1) # take argmax along row   
lr = LogisticRegression(0.1,500)
print(lr)


Untrained MultiClass Logistic Regression Object


In [422]:
x_train_ar.shape

(23536, 186)

In [438]:
%%time
from sklearn.metrics import accuracy_score
x_train_ar=X_train.values
y_target_ar=np.asarray(y_train)
lr = LogisticRegression(.01,10,'newton',.0001)
lr.fit(x_train_ar,y_target_ar)
print(lr)

yhat = lr.predict(x_train_ar)
print('Accuracy of: ',accuracy_score(y_target_ar,yhat))

MultiClass Logistic Regression Object with coefficients:
[[-3.42179002e-07 -3.42179002e-07 -9.74719719e-07 ... -3.24873073e-07
  -3.24450978e-07 -1.77280239e-08]
 [-1.60273891e-07 -1.60273891e-07 -5.96495250e-07 ... -1.48531859e-07
  -1.48124284e-07 -1.21496072e-08]
 [-4.28419043e-05 -4.28419043e-05 -1.37021925e-04 ... -4.03724989e-05
  -4.03196482e-05 -2.52225606e-06]
 [-6.64750903e-07 -6.64750903e-07 -2.14794347e-06 ... -6.28685097e-07
  -6.27844658e-07 -3.69062451e-08]
 [-1.01475511e-06 -1.01475491e-06 -5.73583629e-05 ...  3.33525122e-06
   3.26613571e-06 -4.28089062e-06]
 [-1.66100620e-07 -1.66100620e-07 -5.24855231e-07 ... -1.57596723e-07
  -1.57577783e-07 -8.52283682e-09]]
Accuracy of:  0.34721278042148196
Wall time: 4.22 s


In [440]:
%%time
from sklearn.linear_model import LogisticRegression

lr_sk = LogisticRegression(solver='newton-cg',n_jobs=2,C=.0001, max_iter=10) 
x_train_ar=X_train.values
y_target_ar=np.asarray(y_train)
lr_sk.fit(x_train_ar,y_target_ar)
print(np.hstack((lr_sk.intercept_[:,np.newaxis],lr_sk.coef_)))
yhat = lr_sk.predict(x_train_ar)
print('Accuracy of: ',accuracy_score(y_target_ar,yhat))

[[-3.67380691e-07 -3.66794290e-07  3.83305102e-05 ... -4.08537452e-06
  -3.83871477e-06  3.47192048e-06]
 [ 7.51654247e-06  6.93515648e-06 -2.85991340e-02 ...  4.88418458e-04
   5.51419366e-04 -5.44484210e-04]
 [-9.41664284e-07 -9.40846782e-07 -2.96156727e-06 ... -8.15397438e-07
  -8.12750966e-07 -1.28095816e-07]
 [-3.58179906e-07 -3.56479505e-07 -2.31702188e-04 ... -2.73421503e-05
  -2.72927240e-05  2.69362445e-05]
 [-3.21584353e-07 -3.21465419e-07  2.02699941e-05 ...  1.96504127e-06
   1.92922288e-06 -2.25068830e-06]
 [-2.64656952e-06 -2.59923671e-06  4.50203618e-04 ... -1.35685183e-04
  -1.56286055e-04  1.53686818e-04]]
Accuracy of:  0.4839394969408566
Wall time: 15.3 s


### **2. Modeling**
------

### **3. Deployment**
------

### **4. Optimization Using Mean Squared Error**
------

### **5. References**
------